# Top Martial Characters and Dynasties

After building the kill network in notebook 6 I checked to see if there was an interesting network in taking only those who died in battle. I also wanted to see who had killed the most characters in battle and if there were any characters who had killed only later to be killed themselves in battle. The aggregations match on characters who have the "death_battle" value for "c_d".

In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

## Warriors Details

In [2]:
client = MongoClient()
characters = client.ck2.characters

In [3]:
pipeline = [ 
    {
        "$match" : {"c_d" : "death_battle"}
    },
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"},
        "dyn" : {"$first" : "$killer_dyn.name"},
        "c_d" : {"$first" : "$killer_data.c_d"},
        "d_d" : {"$first" : "$killer_data.d_d"}
    }
    },
    {
        "$project" : { "_id" : 1, "kill_total" : "$kill_total", "name" : {"$concat" : ["$name", " ", "$dyn"]},
                      "died" : "$d_d"}
    },
    {"$sort" : {"kill_total" : -1, 'died' : 1 } },
    {"$limit" : 10}
] 

In [4]:
karl = characters.aggregate(pipeline)

In [5]:
for kill in karl:
    print(kill)

{'_id': 794645, 'kill_total': 4, 'name': 'Mahipala Mahipalid', 'died': datetime.datetime(1231, 3, 6, 0, 0)}
{'_id': 978636, 'kill_total': 4, 'name': 'Samir Samirid', 'died': datetime.datetime(1454, 10, 9, 0, 0)}
{'_id': 532442, 'kill_total': 3, 'name': 'Kormesiy Khunzakhal', 'died': datetime.datetime(864, 8, 23, 0, 0)}
{'_id': 606681, 'kill_total': 3, 'name': 'Buðli af Åland', 'died': datetime.datetime(969, 4, 2, 0, 0)}
{'_id': 673487, 'kill_total': 3, 'name': 'Milosz Dunin', 'died': datetime.datetime(1051, 5, 4, 0, 0)}
{'_id': 655186, 'kill_total': 3, 'name': 'Madanapala Ayudha', 'died': datetime.datetime(1058, 2, 15, 0, 0)}
{'_id': 751264, 'kill_total': 3, 'name': 'Jatanpal Guhilot', 'died': datetime.datetime(1170, 2, 25, 0, 0)}
{'_id': 755938, 'kill_total': 3, 'name': 'Traianos Choirosphaktes', 'died': datetime.datetime(1174, 10, 23, 0, 0)}
{'_id': 812148, 'kill_total': 3, 'name': 'Ereganga Silahara', 'died': datetime.datetime(1230, 5, 9, 0, 0)}
{'_id': 828555, 'kill_total': 3, 'nam

Having a character kill another in battle isn't a very common occurence so racking up 4 kills is impressive. Mahipala served the King of Sindh and Samir was Duke of Moray after the Islamic conquest of British and Irish Isles. Kormesiy Khunzakhal ruled as King of Bavaria and Madanapala Ayudha held 5 Kingdoms in north India.

## Most Militant Dynasties

In [6]:
pipeline = [ 
    {
        "$match" : {"c_d" : "death_battle"}
    },
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_dyn._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_dyn.name"}
    }
    },
    {
        "$project" : { "_id" : 0, "kill_total" : "$kill_total", "name" : "$name"}
    },
    {"$sort" : {"kill_total" : -1 } },
    {"$limit" : 10}
] 

In [7]:
for kill in characters.aggregate(pipeline):
    print(kill)

{'kill_total': 18, 'name': 'Ayudha'}
{'kill_total': 13, 'name': 'Aslamid'}
{'kill_total': 12, 'name': 'Vengi Chalukya'}
{'kill_total': 10, 'name': 'Guhilot'}
{'kill_total': 9, 'name': 'Ay'}
{'kill_total': 9, 'name': 'Penikis'}
{'kill_total': 9, 'name': 'af Munsö'}
{'kill_total': 9, 'name': 'Khunzakhal'}
{'kill_total': 9, 'name': 'Tabghach'}
{'kill_total': 9, 'name': 'Uzur'}


These are the dynasties that have claimed the most kills in battle. The Ayudha rule much of northern India towards the end of the game and the Vengi Chalukya rise to power in the south towards the end, replacing the Ay. 

## Most Blood Spilt

In [8]:
pipeline = [ 
    {
        "$match" : {"c_d" : "death_battle"}
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    {
    "$group": { 
        "_id": "$dnt", 
        "loss_total": { "$sum": 1 },
        "name" : {"$first" : "$dynasty.name"}
    }
    },
    {
        "$project" : { "_id" : 0, "loss_total" : "$loss_total", "name" : "$name"}
    },
    {"$sort" : {"loss_total" : -1 } },
    {"$limit" : 15}
] 

In [9]:
for kill in characters.aggregate(pipeline):
    print(kill)

{'loss_total': 23, 'name': 'Vengi Chalukya'}
{'loss_total': 22, 'name': 'Ayudha'}
{'loss_total': 18, 'name': 'Pratihara'}
{'loss_total': 17, 'name': 'Abbasid'}
{'loss_total': 16, 'name': 'Penikis'}
{'loss_total': 14, 'name': 'Uuidoch'}
{'loss_total': 14, 'name': 'Khunzakhal'}
{'loss_total': 13, 'name': 'Alachisling'}
{'loss_total': 12, 'name': 'Karling'}
{'loss_total': 12, 'name': 'Palemonaitis'}
{'loss_total': 12, 'name': 'Premyslid'}
{'loss_total': 12, 'name': 'Aslamid'}
{'loss_total': 11, 'name': 'Sinda'}
{'loss_total': 11, 'name': 'Skjöldung'}
{'loss_total': 9, 'name': 'Nibelunging'}


These are the dynasties that have lost the most characters to war. 

## Kill and Be Killed

In [11]:
pipeline = [ 
    {
        "$match" : {"c_d" : "death_battle"}
    },
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$match" : {"killer_data.c_d" : "death_battle" }
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"},
        "dyn" : {"$first" : "$killer_dyn.name"},
        "c_d" : {"$first" : "$killer_data.c_d"},
        "d_d" : {"$first" : "$killer_data.d_d"}
    }
    },
    {
        "$project" : { "_id" : 1, "kill_total" : "$kill_total", "name" : {"$concat" : ["$name", " ", "$dyn"]},
                      "died" : "$d_d"}
    },
    {"$sort" : {"kill_total" : -1, "died" : 1 } }
] 

These are all the characters that have both killed and then been killed in battle, ordered by kill total and date of death. Towards the bottom of the list is Conlang de Vannes, the first King of Ireland.

In [12]:
for kill in characters.aggregate(pipeline):
    print(kill)

{'_id': 509768, 'kill_total': 2, 'name': 'Öysteinn Gautske', 'died': datetime.datetime(821, 5, 7, 0, 0)}
{'_id': 639376, 'kill_total': 2, 'name': 'Bataul Khunzakhal', 'died': datetime.datetime(991, 10, 19, 0, 0)}
{'_id': 652251, 'kill_total': 2, 'name': 'Husam Kilabid', 'died': datetime.datetime(1029, 8, 10, 0, 0)}
{'_id': 673131, 'kill_total': 2, 'name': 'Svarnas Wilks', 'died': datetime.datetime(1037, 10, 4, 0, 0)}
{'_id': 707950, 'kill_total': 2, 'name': 'Mukhtar Saadid', 'died': datetime.datetime(1095, 8, 19, 0, 0)}
{'_id': 987971, 'kill_total': 2, 'name': 'Polykarpos Gazis', 'died': datetime.datetime(1416, 12, 13, 0, 0)}
{'_id': 1017103, 'kill_total': 2, 'name': 'Aarif Sulayman', 'died': datetime.datetime(1448, 8, 17, 0, 0)}
{'_id': 992045, 'kill_total': 2, 'name': 'Arunas Palemonaitis', 'died': datetime.datetime(1453, 11, 7, 0, 0)}
{'_id': 510429, 'kill_total': 1, 'name': 'Idris Uyunid', 'died': datetime.datetime(793, 12, 22, 0, 0)}
{'_id': 509682, 'kill_total': 1, 'name': 'Václa